In [1]:
codigo = 'DIRR3'

In [2]:
import pandas as pd
import requests
import json
from base64 import b64encode, b64decode

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# Função para buscar o tradingName pelo ticker
def get_trading_name(ticker):
    params = {"language": "pt-br", "pageNumber": 1, "pageSize": 20, "company": ticker}
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params).decode()
    r = requests.get(
        'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetInitialCompanies/' +
        string,
        verify=False
    )
    for i in r.json()['results']:
        if i['issuingCompany'].lower() == ticker.lower():
            return i['tradingName'].replace('/', '').replace('.', '')
    raise ValueError('Empresa não encontrada')

# Função para obter os proventos
def proventos(ticker):
    tradingName = get_trading_name(ticker)
    results = []
    n = 1
    while True:
        params = {"language": "pt-br", "pageNumber": n, "pageSize": 100, "tradingName": tradingName}
        params = bytes(str(params), encoding="ascii")
        string = b64encode(params).decode()
        r = requests.get(
            'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/' +
            string,
            verify=False
        )
        if not r.json()['results']:
            break
        results += r.json()['results']
        n += 1
    return results


In [3]:
get_trading_name(codigo[:-1])

'DIRECIONAL'

In [4]:
proventos_data = proventos(codigo[:-1])

In [5]:
data = pd.DataFrame(proventos_data)
data.head()

,typeStock,dateApproval,valueCash,ratio,corporateAction,lastDatePriorEx,dateClosingPricePriorExDate,closingPricePriorExDate,quotedPerShares,corporateActionPrice,lastDateTimePriorEx
0,ON,10/12/2025,"1,55",1,DIVIDENDO,16/12/2025,16/12/2025,"16,56",1,"9,359903",2025-12-16T00:00:00
1,ON,16/06/2025,"2,00",1,DIVIDENDO,27/06/2025,27/06/2025,"42,44",1,"4,712535",2025-06-27T00:00:00
2,ON,20/12/2024,"1,27",1,DIVIDENDO,03/01/2025,03/01/2025,"26,56",1,"4,781627",2025-01-03T00:00:00
3,ON,11/09/2024,"0,46",1,DIVIDENDO,17/09/2024,17/09/2024,"33,29",1,"1,381796",2024-09-17T00:00:00
4,ON,01/07/2024,"1,6",1,DIVIDENDO,04/07/2024,04/07/2024,"28,80",1,"5,555556",2024-07-04T00:00:00


In [6]:
data.tail()

,typeStock,dateApproval,valueCash,ratio,corporateAction,lastDatePriorEx,dateClosingPricePriorExDate,closingPricePriorExDate,quotedPerShares,corporateActionPrice,lastDateTimePriorEx
17,ON,30/04/2014,"0,3724405",1,DIVIDENDO,30/04/2014,30/04/2014,"10,88",1,"3,423166",2014-04-30T00:00:00
18,ON,30/04/2013,"0,36386754",1,DIVIDENDO,30/04/2013,30/04/2013,"15,91",1,"2,287037",2013-04-30T00:00:00
19,ON,12/03/2012,"0,2709",1,DIVIDENDO,12/03/2012,12/03/2012,"11,51",1,"2,353606",2012-03-12T00:00:00
20,ON,04/01/2011,"0,301873825",1,DIVIDENDO,05/01/2011,05/01/2011,"13,20",1,"2,286923",2011-01-05T00:00:00
21,ON,26/04/2010,"0,141",1,DIVIDENDO,26/04/2010,26/04/2010,"10,55",1,"1,336493",2010-04-26T00:00:00


In [7]:
import matplotlib.pyplot as plt

# Converte 'valueCash' de string com vírgula para float
data["valueCash"] = data["valueCash"].str.replace(",", ".").astype(float)
# Extrai o ano da data de aprovação
data["year"] = pd.to_datetime(data["dateApproval"], dayfirst=True).dt.year
# Agrupa por ano e soma os proventos
proventos_anuais = data.groupby("year")["valueCash"].sum()


In [9]:
import plotly.graph_objects as go

# Gráfico com Plotly
fig = go.Figure(
    data=[
        go.Bar(
            x=data["year"],
            y=data["valueCash"],
            marker_color="royalblue",
            # text=data["valueCash"].round(3),
            # textposition="outside"
        )
    ]
)

fig.update_layout(
    title="Proventos Pagos por Ano",
    xaxis_title="Ano",
    yaxis_title="Total de Proventos (R$)",
    xaxis_tickangle=90,
    template="plotly_white",
    height=500
)

fig.show()


In [10]:
# proventos via Fundamentus

from io import StringIO

url1 = f'https://www.fundamentus.com.br/proventos.php?papel={codigo}&tipo=2'
header = { 
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36", "X-Requested-With": "XMLHttpRequest"
    }
r1 = requests.get(url1, headers=header)
dy = pd.read_html(StringIO(r1.text), decimal=',', thousands='.')[0]
dy['Data'] = pd.to_datetime(dy['Data'] , dayfirst=True)
dy.set_index('Data',inplace =True )

dy['Valor/acao'] = dy['Valor']/dy['Por quantas ações']

fig = go.Figure(
    [ go.Bar(x=dy.index.year , y = dy['Valor'],name='Valor',
            textposition="outside"),
     go.Bar(x=dy.index.year , y = dy['Valor/acao'],name='Valor/ação',
            textposition="outside")],
    layout=dict(title='Dividendos ',height=500, width=700 ))
fig.show()

testes:

In [11]:
s = 'eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjIwLCJ0cmFkaW5nTmFtZSI6IkJSQVNJTCJ9'
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/'
r=requests.get(url + s)
r.json()

{'page': {'pageNumber': 1,
  'pageSize': 20,
  'totalRecords': 268,
  'totalPages': 14},
 'results': [{'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,04583263233',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '02/12/2025',
   'dateClosingPricePriorExDate': '02/12/2025',
   'closingPricePriorExDate': '22,56',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,203159',
   'lastDateTimePriorEx': '2025-12-02T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,07192713139',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '01/12/2025',
   'dateClosingPricePriorExDate': '01/12/2025',
   'closingPricePriorExDate': '22,26',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,323123',
   'lastDateTimePriorEx': '2025-12-01T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '14/05/2025',
   'valueCash': '0,33425840109',
   'ratio': '1',
   'corporateAction': 'JR

In [12]:
padrao = b64decode(s)
padrao

b'{"language":"pt-br","pageNumber":1,"pageSize":20,"tradingName":"BRASIL"}'

In [13]:
# max item/pagina = 120
s = b64encode(b'{"language":"pt-br","pageNumber":1,"pageSize":120,"tradingName":"BRASIL"}').decode()
s

'eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwidHJhZGluZ05hbWUiOiJCUkFTSUwifQ=='

In [14]:
r=requests.get(url + s)
print(r.json()['results'][-1])
r.json()

{'typeStock': 'ON', 'dateApproval': '11/08/2015', 'valueCash': '0,00032874943', 'ratio': '1', 'corporateAction': 'RENDIMENTO', 'lastDatePriorEx': '21/08/2015', 'dateClosingPricePriorExDate': '21/08/2015', 'closingPricePriorExDate': '18,56', 'quotedPerShares': '1', 'corporateActionPrice': '0,001771', 'lastDateTimePriorEx': '2015-08-21T00:00:00'}


{'page': {'pageNumber': 1,
  'pageSize': 120,
  'totalRecords': 268,
  'totalPages': 3},
 'results': [{'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,04583263233',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '02/12/2025',
   'dateClosingPricePriorExDate': '02/12/2025',
   'closingPricePriorExDate': '22,56',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,203159',
   'lastDateTimePriorEx': '2025-12-02T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,07192713139',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '01/12/2025',
   'dateClosingPricePriorExDate': '01/12/2025',
   'closingPricePriorExDate': '22,26',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,323123',
   'lastDateTimePriorEx': '2025-12-01T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '14/05/2025',
   'valueCash': '0,33425840109',
   'ratio': '1',
   'corporateAction': 'JR

Fonte: https://fabiomt.medium.com/como-obter-o-hist%C3%B3rico-de-dividendos-e-outros-proventos-com-python-direto-do-site-da-b3-90a507623418
